In [ ]:
import pandas as pd
import numpy as np
import json

import os
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from keras.layers import Dense, Embedding, Input, Flatten, Dropout, concatenate, BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential, Model
#from keras imfrom keras.utils import to_categoricalport backend as K
from keras import regularizers
#from math import log2,ceil
from sklearn.metrics import roc_auc_score

from keras import backend as K
from math import log

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [ ]:
#Read dataset
from data_access_sdk_python.reader import DataSetReader
from datetime import date
reader = DataSetReader()
df0 = reader.load(data_set_id="5d3820dd5e44d414521d032d", ims_org="56076ACB5CC99C9F0A495C99@AdobeOrg", batch_id="3fb05a1602f24a1d8f4b2087c28d0dbc")
df0.head()

for index,row in df0.iterrows():
    if row['21'] == '1':
        row['21'] =0
    else:
        row['21'] =1

        
credit = df0.apply(pd.to_numeric)

INFO:DataAccessSDK:There are 1 file(s) in this dataset/batch, limit_files: all
INFO:azure.datalake.store.core:closing stream
INFO:DataAccessSDK:#1/1 batch file platform/5d3820dd5e44d414521d032d/datasetViewId=5d3820dd5e44d414521d032e/batchId=3fb05a1602f24a1d8f4b2087c28d0dbc/part-00000-40ff4344-4255-44a7-b486-0f3787d3bc70-c000.snappy.parquet : 7400 rows read. 371.73 MB memory used for this process.
INFO:DataAccessSDK:7400 rows total in dataframe


In [ ]:
from sklearn import preprocessing

import numpy as np
import pandas as pd
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
import time


SAMPLED =True
sample = [4,7,10]
Gmean=[]
Gstd=[]
rocmean=[]
rocstd=[]

for s in range(30):
    sample_frac= sample[s]
    gmean_30=[]
    roc_30=[]
    y_col = '21'
    for i in range(30):
        seed = 42
        random.seed(42)
        tf.set_random_seed(seed)
        np.random.seed(seed)
        sns.set_style("whitegrid")

        print("Imports done..")

        onehot_encoder = OneHotEncoder(sparse=False)


        SAMPLED = True
        ALPHA =0.2


        col_others=[]

        credit =credit.sample(frac=1)


  
        X = credit.drop(labels=[y_col], axis='columns')

        for i in X.columns.tolist():

            col_others.append(i)


        for i in col_others:

            x = X[i].values #returns a numpy array
            x=x.reshape(-1,1)
            min_max_scaler = preprocessing.MinMaxScaler()
            x_scaled = min_max_scaler.fit_transform(x)

            X[i] = x_scaled

        X= pd.get_dummies(X)
  
        y = credit[y_col].as_matrix()
       

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)#,random_state=seed)


        sample_frac =sample_frac
        pre_x_train = X_train
        pre_x_train['y'] = y_train

        pre_x_train1 = pre_x_train.loc[pre_x_train['y'] == 1]
        pre_x_train0 = pre_x_train.loc[pre_x_train['y'] == 0]

        if(pre_x_train1.shape[0] > pre_x_train0.shape[0]):

            df_elements = pre_x_train0.sample(n=sample_frac)

            bigdata = pre_x_train1.append(df_elements, ignore_index=True)
        else:

            df_elements = pre_x_train1.sample(n=sample_frac)
      
            bigdata = pre_x_train0.append(df_elements, ignore_index=True)

        bigdata =bigdata.sample(frac=1)
        pre_x_train = bigdata.drop(labels=['y'], axis='columns')

        pre_y_train = bigdata['y'].as_matrix()
        X_resampled, y_resampled =pre_x_train,pre_y_train
        X_train = X_resampled
        y_train =y_resampled

        X_train_dropped = X_train
        print (X_train_dropped.shape)
        X_test_dropped = X_test
        print (X_test_dropped.shape)




        def uniform_draw_feat_class(data_df, target_name, draw_size):
            """ Sample out 1 particular class out of the full data uniformly in a predefined sample size,
            segregate pandas dataframe into their respective target classes. """

            # Break down full dataset into their respective classes
            target_class_dfs = {}
            n_classes = data_df[target_name].unique()

            for unique_class in n_classes:
                # Store each segregated class dataframes as dictionary
                class_i_df = data_df[data_df[target_name] == unique_class]
                print (class_i_df.columns)
                class_i_df = (class_i_df.reset_index()).drop(columns=['index'])
                target_class_dfs[unique_class] = class_i_df

            # Obtain uniform samples of size 'draw_size' and store as dictionary
            uniform_class_sample_draws = {}

            # Pandas df. sampling default None equivalently uniform draws
            for unique_class in n_classes:
                unif_samples = target_class_dfs[unique_class].sample(n=draw_size, replace=True)
                sample_df = pd.DataFrame(unif_samples, columns=data_df.columns)
                uniform_class_sample_draws[unique_class] = unif_samples

            # Returns dictionary of pandas dataframes, split into respective classes
            return uniform_class_sample_draws


        # In[5]:


        def model(input_data, num_classes=2):
            fc_1 = tf.layers.dense(input_data, 132)#, #name='fc_1')
            fc_1 = tf.nn.relu(fc_1)#, name='fc_1_relu')

            fc_2 = tf.layers.dense(fc_1, 64)#, name='fc_2')
            fc_2 = tf.nn.relu(fc_2)#, name='fc_2_relu')


            fc_4 = tf.layers.dense(fc_2, num_classes)#, name='fc_4')                                                                                                                                                                                       
            #fc_4 = tf.nn.sigmoid(fc_4, name='fc_4_relu')

            return fc_4

        def get_loss(preds, labs):
            loss_val = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=preds, labels=labs))
            return loss_val



        def get_accuracy(preds_list, gt_list):
            num_samples = gt_list.shape[0]
            ct = 0
            for ix in range(num_samples):
                if np.argmax(preds_list[ix]) == np.argmax(gt_list[ix]):
                    ct+=1

            return ct/float(num_samples)

        def get_acc_perlabel(preds_list, gt_list):
                num_samples = gt_list.shape[0]
                ct_index = [0, 0]
                acc = [0, 0]
                for ix in range(num_samples):
                        pred_index = np.argmax(preds_list[ix])
                        gt_index = np.argmax(gt_list[ix])
                        ct_index[gt_index]+=1
                        if pred_index == gt_index:
                                acc[gt_index]+=1

                return acc[0]/float(ct_index[0]), acc[1]/float(ct_index[1])



        def generate_data(X_train_dropped, X_test_dropped, y_train, y_test):
                X_train_val = X_train_dropped.values
                X_test_val = X_test_dropped.values

                y_train_val = onehot_encoder.fit_transform(np.reshape(y_train, (y_train.shape[0], 1)))
                y_test_val = onehot_encoder.fit_transform(np.reshape(y_test, (y_test.shape[0], 1)))

                return X_train_val, y_train_val, X_test_val, y_test_val


        def transformed_sample(vec1, vec2, alpha):
                x_ret = alpha*vec1 + (1-alpha)*vec2
                y_ret = np.array([alpha, 1-alpha])

                return x_ret, y_ret


        def get_train_samples_by_label(x_train, y_train):
                lab_0_list = []
                lab_1_list = []
                for item in range(len(x_train)):

                        if int(y_train[item]) == 0:
                                lab_0_list.append(x_train[item])
                        elif int(y_train[item]) == 1:
                                lab_1_list.append(x_train[item])

                return lab_0_list, lab_1_list


        def entropy_calc(vec,zer):
            a = vec[0]
            b = vec[1]
            #print (a,b)
            try:
                if (vec[0]==1):
                        prob = 0
                        return prob
                elif(vec[1]==1):
                        prob =0
                        return prob
                else:
                    prob = -(a*log(a,2)) - (b*log(b,2))
            except ValueError:
                print(a,b)
                prob=0

            return prob

        def get_train_samples_by_label_pred(x_train, y_train,pred_list):
             
                lab_0_list = []
                weight0 =[]
                weight0R=[]
                lab_1_list = []
                weight1 =[]
                weight1R =[]
                sum0=0
                sum0R=0
                sum1=0
                sum1R = 0
                print (y_train.shape, x_train.shape)
                print (type(x_train))
                for item in range(len(x_train)):
       
                    if int(y_train[item]) == 0:
              
                            entropy =entropy_calc(pred_list[item],True)

                            lab_0_list.append(np.append(x_train[item],entropy))
                            sum0 =sum0+entropy
                            sum0R = sum0R+(1-entropy)
                    elif int(y_train[item]) == 1:
                            entropy =entropy_calc(pred_list[item],False)
                            lab_1_list.append(np.append(x_train[item],entropy))
                            sum1 =sum1+entropy
                            sum1R=sum1R+(1-entropy)
                lab_0_list=np.array(lab_0_list)
                lab_1_list=np.array(lab_1_list)
                #print('lab0list',lab_0_list)
                lab_0_list=lab_0_list[lab_0_list[:,-1].argsort()]
                #print('lab0list',lab_0_list)
                lab_1_list=lab_1_list[lab_1_list[:,-1].argsort()]
                for i in range(len(lab_0_list)):
                    weight0.append(lab_0_list[i][-1]/sum0)
                    weight0R.append((1-lab_0_list[i][-1])/sum0R)
                for i in range(len(lab_1_list)):
                    weight1.append(lab_1_list[i][-1]/sum1)
                    weight1R.append((1-lab_1_list[i][-1])/sum1R)
                #print ('weight0',weight0)
                return lab_0_list[:,:-1], lab_1_list[:,:-1],weight0,weight1,weight0R,weight1R


        def input_vecs(list1, list2):
                idx1 = np.random.randint(0, len(list1))
                idx2 = np.random.randint(0, len(list2))


                return np.array(list1[idx1]), np.array(list2[idx2])

        def input_vecs_pred(list1, list2,weight0,weight1,weight0R,weight1R,turn):

            if(turn == 0):
                #both from near the boundry
                from numpy.random import choice
                #print('weight0',weight0)
                draw0 = choice(len(list1), 1,weight0)
                draw1 = choice(len(list2), 1,weight1)
                row0 = list1[draw0].flatten()
                row1 = list2[draw1].flatten()
                return row0,row1
            if(turn == 1):
                #both away from boundary
                from numpy.random import choice

                draw0 = choice(len(list1), 1,weight0R)
                draw1 = choice(len(list2), 1,weight1R)
                row0 = list1[draw0].flatten()
                row1 = list2[draw1].flatten()
                return row0,row1
            if(turn == 2):
                #one near one far
                from numpy.random import choice
                r = random.uniform(0, 1)
                if(r<0.5):

                    #print (weight1R)
                    draw0 = choice(len(list1), 1,weight0)
                    draw1 = choice(len(list2), 1,weight1R)
                    row0 = list1[draw0].flatten()
                    row1 = list2[draw1].flatten()
                else: 

                    draw0 = choice(len(list1), 1,weight0R)
                    draw1 = choice(len(list2), 1,weight1)
                    row0 = list1[draw0].flatten()
                    row1 = list2[draw1].flatten()


                return row0,row1


        def get_label_vectors(x_train, y_train):
                #print ("x_train shape: ", x_train.shape)
                lab0 = []
                lab1 = []
                num_samples = x_train.shape[0]
                for item in range(num_samples):
                        lab = y_train[item]

        # def auc(y_true, y_pred):
        #      return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

        def auc(y_true, y_pred):
            return( roc_auc_score(y_true, y_pred))


        def gmean(y_true, y_pred):
            from sklearn.metrics import confusion_matrix
            y_true = np.argmax(y_true, axis=1)
            y_pred = np.argmax(y_pred, axis=1)
            TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()
            tp = TP/(TP+FN)
            tn = TN/(TN+FP) 
            gmean = np.sqrt(tp*tn)
            return gmean



        def unmatched_select(x_train_preds,y_train_val):
            match =0
            unmatched_rows=[]
            for i in range (len(x_train_preds)):
                predicted_list = x_train_preds.argmax(axis=1)
                real_list = y_train_val.argmax(axis=1)
                if (predicted_list[i] ==real_list[i]):

                        match = match+1
                else:
                    unmatched_rows.append(X_train_val[i])

            print ('matched rows',match/len(x_train_preds))
            return unmatched_rows

        def max_all(array):
            max_all =[]
            for i in array:
                max_all.append(np.amax(i))
            return max_all

        def generate_data_sampled(X_resampled, X_test_dropped, y_resampled, y_test,num_gen_samples,unmatched,i,pred,new_samp_x,new_samp_y,turn,val):
                if(i==0):
                    x_test_val = X_test_dropped.values
                    y_test_val = onehot_encoder.fit_transform(np.reshape(y_test, (y_test.shape[0], 1)))

                    curr_y_train_val = onehot_encoder.fit_transform(np.reshape(y_resampled, (y_resampled.shape[0], 1)))
                    curr_x_train_val = X_resampled.values
                    return np.array(curr_x_train_val), np.array(curr_y_train_val), x_test_val, y_test_val

                lab_0_list, lab_1_list,weight0,weight1,weight0R,weight1R = get_train_samples_by_label_pred(X_resampled.values, y_resampled,pred)



                x_test_val = X_test_dropped.values
                y_test_val = onehot_encoder.fit_transform(np.reshape(y_test, (y_test.shape[0], 1)))

                curr_y_train_val = onehot_encoder.fit_transform(np.reshape(y_resampled, (y_resampled.shape[0], 1)))
                curr_x_train_val = X_resampled.values

                num_train_samples = X_resampled.values.shape[0]

        #         train_samples = []
                x_train_samples = []
                y_train_samples = []
                for item in range(num_train_samples):
                        x_train_samples.append(curr_x_train_val[item])
                        y_train_samples.append(curr_y_train_val[item])
                        #train_samples.append([curr_x_train_val[item], curr_y_train_val[item]])

                print("Generating new samples..")
                ## Generate new samples
                if(i!=0):
                    for ctr in range(int(X_resampled.shape[0]*0.5)):
       

                            vec1, vec2= input_vecs_pred(lab_0_list, lab_1_list,weight0,weight1,weight0R,weight1R,val)

             

                            ratio = np.random.beta(0.5,0.5)
 
                            if(turn==0):
                                new_x, new_y = transformed_sample(vec1, vec2, ratio)
                            

                            new_samp_x.append(new_x)
                            new_samp_y.append(new_y)

                    x_train_samples=np.concatenate((np.array(x_train_samples),np.array(new_samp_x)),axis=0)
                    y_train_samples=np.concatenate((np.array(y_train_samples),np.array(new_samp_y)),axis=0)

                return np.array(x_train_samples), np.array(y_train_samples), x_test_val, y_test_val
        def collect_vals(arr):
            ar =[]
            for i in arr:
                ar.append(i[-1])
            return ar

        def collect_val(arr):
            ar =[]
            for i in arr:
                ar.append(np.amax(i))
            return ar

        def gen_samp(X_train,y_train,num,type_sampler):

            if(type_sampler == 1):
                curr_y_train_val = onehot_encoder.fit_transform(np.reshape(y_train, (y_train.shape[0], 1)))
                if(num>0):

                    from imblearn.over_sampling import RandomOverSampler
                    ros = RandomOverSampler(random_state=0)
                    X_resampled, y_resampled = ros.fit_resample(X_train, curr_y_train_val)
       
                else:
                    X_resampled, y_resampled = X_train, y_train
            if(type_sampler == 2):
                curr_y_train_val = onehot_encoder.fit_transform(np.reshape(y_train, (y_train.shape[0], 1)))
                if(num>0):

                    from imblearn.over_sampling import SMOTE, ADASYN
                    X_resampled, y_resampled = SMOTE().fit_resample(X_train, curr_y_train_val)
  
                else:
                    X_resampled, y_resampled = X_train, y_train
            if(type_sampler == 3):
                curr_y_train_val = onehot_encoder.fit_transform(np.reshape(y_train, (y_train.shape[0], 1)))
                if(num>0):

                    from imblearn.over_sampling import SMOTE, ADASYN
                    X_resampled, y_resampled = ADASYN().fit_resample(X_train, curr_y_train_val)
         
                else:
                    X_resampled, y_resampled = X_train, y_train



            return X_resampled, y_resampled



        if not SAMPLED:
              print ('NOT SAMPLED ')

        else:


            turn=0
            final_roc=[]
            final_gmean=[]
            val=0
      #for linear mixup, near-near entropy 
            while(val<1):
                arr_f=[]
                pred_val=[]
                yt_val=[]
                arr_1f=[]
                g_m=[]
                arr_0f=[]
                unmatched=[]
                new_samp_x=[]
                new_samp_y=[]
                pred =[]
                for i in range(20):
                    arr=[]
                    g_mean=[]
                    arr_1=[]
                    arr_0=[]

                    r_val=0
                    
                    X_train_val, y_train_val, X_test_val, y_test_val = generate_data_sampled(X_resampled, 
                                                                                              X_test_dropped, 
                                                                                              y_resampled, 
                                                                                              y_test,
                                                                                              int(X.shape[0]*i),
                                                                                              unmatched,i,
                                                                                              pred,
                                                                                              new_samp_x,
                                                                                              new_samp_y,
                                                                                              turn%2,
                                                                                              val%3)


                    '''FLAGS SETUP'''
                    feat_len = X_train_dropped.shape[-1]


                    num_classes = np.unique(y_test).shape[0]

                    gan_epochs =300

                    mini_bat = 500 # Equivalently gan_fraudless/fraud.batch_

                    num_train_samples = X_train_val.shape[0]

                    seed = 42
                    random.seed(seed)
                    tf.set_random_seed(seed)
                    np.random.seed(seed)

                    LR = 0.0001

                    print("Building model...")

         

                    print("Now starting..\n")

                    input_place = tf.placeholder(shape=[None, feat_len] ,dtype=tf.float32)
                    label_place = tf.placeholder(shape=[None, num_classes], dtype=tf.float32)
                    vec = model(input_place)
                    out_vec = tf.nn.sigmoid(vec)# name='vec_sigmoid')
                    loss = get_loss(vec, label_place)



                    var_list = tf.trainable_variables()[-8:]
                    print ("Num trainable: ", len(var_list))

                    gradients = tf.gradients(loss, var_list)

                    grads_and_vars = list(zip(gradients, var_list))
                    #print(grads_and_vars)
                    optimizer = tf.train.AdamOptimizer(learning_rate = LR)
                    train_op = optimizer.apply_gradients(grads_and_vars=grads_and_vars)
                    print ('optimizer setup..')

                    sess = tf.InteractiveSession()

                    sess.run(tf.global_variables_initializer())

                    for epoch in range(gan_epochs+1):
                        itr = 0
                        epoch_loss = 0.0
                        while itr*mini_bat<num_train_samples:

                            x = X_train_val[itr*mini_bat:itr*mini_bat+mini_bat]
                            y = y_train_val[itr*mini_bat:itr*mini_bat+mini_bat]

                            _, loss_val = sess.run([train_op, loss], feed_dict={input_place:x, label_place: y})

                            epoch_loss+=loss_val
                            itr+=1

                        epoch_loss = epoch_loss/float(itr)
                        #print("epoch loss: ", epoch_loss)

                        if epoch%50 == 0 and epoch>=1:

    #                         print("Testing at Epoch: ", epoch)
                            x_test_preds = sess.run(out_vec, feed_dict={input_place:X_test_val})
                            acc = get_accuracy(x_test_preds, y_test_val)
                            lab0_acc, lab1_acc = get_acc_perlabel(x_test_preds, y_test_val)
                             #print (x_test_preds,y_test_val)

                            g_mean_f = gmean(y_test_val,x_test_preds)
                            auc_roc_f = auc(y_test_val,x_test_preds)
                            from sklearn.metrics import roc_curve


                            r_val = auc_roc_f
                            arr.append(r_val)
                            arr_1.append(lab1_acc)
                            arr_0.append(lab0_acc)
                            g_mean.append( g_mean_f)

                    x_train_preds = sess.run(out_vec, feed_dict={input_place:np.array(X_train_dropped)})

                    pred = x_train_preds
                    print ('x_train_preds',len(x_train_preds))
 
                    from sklearn.metrics import roc_curve
                    import matplotlib.pyplot as pyplot
                    yt_val.append(y_test_val.argmax(axis=1))
                    pred_val.append(x_test_preds.argmax(axis=1))



                    arr_f.append(arr)
                    arr_1f.append(arr_1)
                    arr_0f.append(arr_0)
                    g_m.append(g_mean)
                    sess.close()
                    tf.reset_default_graph()


                ar_roc = collect_val(arr_f)
                final_roc.append(ar_roc)
                final_gmean.append(collect_val(g_m))

                turn = turn+1
                val=val+1

            gmean_30.append(max_all(final_gmean))
            roc_30.append(max_all(final_roc))
            print('---------------------------------')
            
    def mean_std(array,mean,std):

        mean.append(np.mean(array))
        std.append(np.std(array))
                        
    mean_std(gmean_30,Gmean,Gstd)
    mean_std(roc_30,rocmean,rocstd)
                





INFO:matplotlib.font_manager:generated new fontManager
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/sit

Imports done..
0    3736
1    3664
Name: 21, dtype: int64
(1910, 20)
(3700, 20)
Building model...
Now starting..



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Num trainable:  6
optimizer setup..
x_train_preds 1910
(1910,) (1910, 20)
<class 'numpy.ndarray'>
Generating new samples..
Building model...
Now starting..



/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

...

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

Num trainable:  6
optimizer setup..
x_train_preds 1919
(1919,) (1919, 20)
<class 'numpy.ndarray'>
Generating new samples..


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

Building model...
Now starting..



Num trainable:  6
optimizer setup..
x_train_preds 1919
---------------------------------
